# <span style="color:#039590">  One Word Transcriptions  (OWT)

All the prior transcriptions are wrong since an old function to extract the correct data frame was used.

This script gets the correct transcriptions for all the audio files.



## <span style="color:#4BBC8E"> Library Imports

In [1]:
from transformers import pipeline, AutoModel, AutoTokenizer
import pandas as pd 
import numpy as np 
import warnings
import whisper
import torch
import jiwer
import os

2024-07-05 13:23:03.235310: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-05 13:23:04.214142: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## <span style="color:#9BDE7E"> Function definitions

Functions for updating the models, ensure that saved CSV files is not over written, and the correct whisper path is defined.

In [2]:
# Function to make a filename that does not exist in the directory
def get_new_csv_name(directory, base_name):
    test_for = os.path.join(directory, base_name + '.csv')
    version = 1
    csv_name = os.path.join(directory, f"{base_name}_v{version}.csv")
    # Check for existing versions and increment until a new name is found
    while os.path.isfile(csv_name):
        version += 1
        csv_name = os.path.join(directory, f"{base_name}_v{version}.csv")
    return csv_name, version

# Functions to get proper whisper path
def get_whisper_path(name):
    # Handle difference between NB and standard whisper
    if name.startswith("nb"): # Put correct filepath 
        path = 'NbAiLabBeta/' + name    
    else: 
        path = 'openai/whisper-' + name
    return path

def update_model(name):
    # pip install --upgrade transformers datasets huggingface_hub
    # warnings.filterwarnings(action='ignore', category=FutureWarning, message=r'.*`resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`*')
    warnings.filterwarnings(action='ignore', category=FutureWarning, message=r'.*`resume_download` is deprecated and will be removed in version 1.0.0*')
    model = AutoModel.from_pretrained(name, force_download=True) # Example for loading a model
    tokenizer = AutoTokenizer.from_pretrained(name, force_download=True) # Example for loading a tokenizer
    return model, tokenizer

# Function for updating all the models in a list
def update_all_models(model_list):
    for model in model_list:
        if model.startswith("nb"):
            model = 'NbAiLabBeta/' + model
            update_model(model)    
        else:
            model = 'openai/whisper-' + model 
            update_model(model)

## <span style="color:#F9F871"> Data Initialization </span> and <span style="color:#F9F871"> Model update

In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

data_path = '/talebase/data/speech_raw/teflon_no/'
path_all_audio = data_path + 'speech16khz/'

# Data frame fixing -------------
df_new_assessment = pd.read_csv(data_path + 'assessments.csv') # Loade file
df_new_no_zero = df_new_assessment[df_new_assessment['Score'] != 0] # Remove Zero scores

# Get the index of the max score for each file name group
max_score_indices = df_new_no_zero.groupby('File name')['Score'].idxmax() 
df_new_no_dup = df_new_no_zero.loc[max_score_indices]

# Get the mean score for each file name group
mean_scores = df_new_no_zero.groupby('File name')['Score'].mean().apply(np.ceil).astype(int)

# Replace the 'Score' column with the rounded mean scores
df_new_no_dup['Score'] = df_new_no_dup['File name'].map(mean_scores)
df_new_no_dup.reset_index(drop=True, inplace=True)
df_assessment = df_new_no_dup 

In [4]:
model_names = ['tiny', 'nb-whisper-tiny', 'nb-whisper-tiny-verbatim',
                'base', 'nb-whisper-base', 'nb-whisper-base-verbatim',
                'medium', 'nb-whisper-medium', 'nb-whisper-medium-verbatim']

# update_all_models(model_names)

In [5]:
update_all_models(model_names)

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## <span style="color:#B8E067"> Transcription </span> and <span style="color:#B8E067"> CSV saving

In [5]:
for model_name in model_names:
    
    # --------  MODEL SETUP -------- #
    model_name_path = get_whisper_path(model_name)
    model = pipeline("automatic-speech-recognition", model_name_path) # Load the model
    print(f"Loaded the whisper model {model_name}")
    
    model_transcriptions = pd.DataFrame(columns=['File name', 'Word', 'Transcribed', 
                                'CER (Character Error Rate)', 'OG Score'])  
    
    # -------- TRANSCRIPTION --------  #
    for i, row in df_assessment.iterrows(): # Iterate over the data frame
        audio_path = path_all_audio + row['File name']
        transcription = model(audio_path, generate_kwargs={'task': 'transcribe', 'language': 'no'}) 
        
        # Checks for empty strings in transcriptions
        if 'text' in transcription and transcription['text'].strip(): 
            # -------- Calculate CER -------- #
            cer = jiwer.cer(transcription['text'], row['Word'])
            
            model_transcriptions.loc[i] = [row['File name'], 
                                        row['Word'], transcription['text'], 
                                        cer, row['Score']] # Directly add a new row to the DataFrame    
        # else: # If the transcription is empty -->--> handle_empty_translations.ipynb <--<--
            
            
        # if i == 2: # Used to test if the files are saved whit the correct name
        #     index_empty_rows = [1, 2, 3] 
        #     break  # Quick test to see if the code works
        
        # Have consistent prints so the server does not disconnect
        if i % 900 == 0: print(f'Index {i} of {len(df_assessment)}') 
        
    print(f"Finished transcribing all the words in {model_name}\n")
    
    # -------- Save CVS -------- #
    try: 
        # CVS name
        transcription_dir = '../Transcriptions'
        base_name = f'true_transcription_{model_name}'
        csv_file_name, number = get_new_csv_name(transcription_dir, base_name)
        # Use ""./Final Scripts/handle_empty_translations.ipynb" To get the empty transcriptions
        
        model_transcriptions.to_csv(csv_file_name, index=False) 
        
    except Exception as e: # If it can't save it for some reason print the df
        print(f'Error saving the transcribed whisper model {model_name} to csv: {e}') 
    # break


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Index 0 of 9322
Index 900 of 9322
Index 1800 of 9322
Index 2700 of 9322
Index 3600 of 9322
Index 4500 of 9322
Index 5400 of 9322
Index 6300 of 9322
Index 7200 of 9322
Index 8100 of 9322
Index 9000 of 9322
Finished transcribing all the words in medium



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Index 0 of 9322
Index 900 of 9322
Index 1800 of 9322
Index 2700 of 9322
Index 3600 of 9322
Index 4500 of 9322
Index 5400 of 9322
Index 6300 of 9322
Index 7200 of 9322
Index 8100 of 9322
Index 9000 of 9322
Finished transcribing all the words in nb-whisper-medium



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Index 0 of 9322
Index 900 of 9322
Index 1800 of 9322
Index 2700 of 9322
Index 3600 of 9322
Index 4500 of 9322
Index 5400 of 9322
Index 6300 of 9322
Index 7200 of 9322
Index 8100 of 9322
Index 9000 of 9322
Finished transcribing all the words in nb-whisper-medium-verbatim

